In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic
from python.fluorophores import FlCollection
from python.psfs import PsfVectorial
from python.estimators import est_qLSQiter2D
from python.estimators import backgroundsubtractor
from python.simulators import Simulator

## Investigate the effect of background
Causes: imperfect zero, autofluorescence background, nearby fluorophore
XXX background handling of estimator wrong, this makes this not well
interpretable. Use bg calibration and background in estimator

In [ ]:
psf_vec = PsfVectorial()
psf_vec.setpinhole(AU=1)

In [ ]:
sim = Simulator()
laserpower = 10
numberOfLocalizations = 1000
L = 75
probecenter = True
estimatebackground = True
sim.definePattern("donut", psf_vec,
                  phasemask="vortex", 
                  makepattern="orbitscan", 
                  orbitpoints=4, 
                  probecenter=probecenter,
                  orbitL=L,
                  laserpower=laserpower,
                  pointdwelltime=0.1)
sim.defineComponent("estdonut", "estimator", est_qLSQiter2D, parameters=[L, probecenter], dim=(0,1))
sim.defineComponent("bg", "background", backgroundsubtractor, parameters=["background_estimated"])
seq = ["donut", "bg", "estdonut"]
fltestpos = [10, 0, 0]
fl1 = FlStatic()
fl1.pos = fltestpos
sim.fluorophores = fl1


In [ ]:
plt.figure()

In [ ]:
# no background
sim.background_estimated = 0
fl1.pos = fltestpos
psf_vec.zerooffset = 0
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("no background")
sim.summarize_results(out)

# imperfect zero

In [ ]:

psf_vec.zerooffset=0.01
# estimate background by scanning centered fluorophore
fl1.pos = [0, 0, 0]
sim.background_estimated = 0
outtest = sim.runSequence(seq, maxlocs=numberOfLocalizations)
bgphot = np.mean(outtest.raw[:,-1])
bgzeroscan = bgphot/outtest.par[0].pointdwelltime
bgestdirect=fl1.brightness*psf_vec.zerooffset*laserpower*psf_vec.PSFs["pinhole63300"].interp([0, 0, 0])   # illustration how zero offset is converted into background
sim.background_estimated=bgzeroscan
fl1.pos=fltestpos
out=sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("zerooffset: ")
sim.summarize_results(out)

## Autofluorescence background

In [ ]:
psf_vec.zerooffset = 0.0
sim.background = 30
sim.background_estimated = sim.background*laserpower  # in general, the GT background is not known but needs to be calibrated 
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("fluorescence background: ")
sim.summarize_results(out)

## nearby fluorophore

In [ ]:
fl2 = FlStatic()
fl2.pos = [50, 50, 400]

fc = FlCollection()
fc.add([fl1, fl2])
psf_vec.zerooffset = 0
sim.fluorophores = fc  # make a simulator and attach fluorophore
sim.background_estimated = 0  # we don't know bg
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("nearby fluorophore: ")
sim.summarize_results(out)